# RPGゲームを作ろう

### ・セクション1: キャラクターClassを作ろう

In [24]:
#クラスの作成
class Character:
    def __init__(self, name, hp, power):
        self.name = name
        self.hp = hp
        self.power = power
    
    def attack(self, other):
        other.hp -= self.power
        return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。"

In [25]:
#インスタンスの生成
hero = Character("勇者", 100, 15)
monster = Character("スライム", 50, 10)

In [27]:
#戦闘
print(hero.attack(monster))
print(monster.hp)

勇者がスライムを攻撃！スライムの体力は20になった。
20


In [53]:
"""
練習問題1) キャラクターにmpを追加しよう。また、attackメソッドを変更して、体力が0より小さくならないようにしよう
"""

class Character:
    def __init__(self, name, hp, power, mp):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.power = power
        self.mp = mp
    
    def attack(self, other):
        other.hp -= self.power
        if other.hp <= 0:
            other.hp = 0
            return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
        return f"{self.name}が{other.name}を攻撃！{other.name}の体力は{other.hp}になった。"

    #生存チェック
    def is_alive(self):
        return self.hp > 0

In [39]:
#インスタンスの生成
hero = Character("勇者", 100, 15, 10)
monster = Character("スライム", 50, 10, 5)

for i in range(4):
    print(hero.attack(monster))

勇者がスライムを攻撃！スライムの体力は35になった。
勇者がスライムを攻撃！スライムの体力は20になった。
勇者がスライムを攻撃！スライムの体力は5になった。
勇者がスライムを攻撃！スライムの体力は0になった。
スライムは勇者に倒された!


---
###  ・セクション2: キャラクターの派生クラスの冒険者クラスを作ろう

In [54]:
class Explorer(Character):
    def __init__(self, name, hp, power, mp, role, level=1):
        super().__init__(name, hp, power, mp)
        self.role = role
        self.level = level
        self._initial_state = copy.deepcopy(self.__dict__)

    #スペシャルアタックを追加
    def special_attack(self, other):
        if self.mp < 3:
            return f"MPが足りない！ (現在のMP{self.mp})"          
        #勇者の必殺技
        if self.role == HERO:
            if not isinstance(other, Explorer):
                other.hp -= self.power*2
                self.mp -= 3  
                if other.hp <= 0:
                    other.hp = 0
                    return f"{self.name}の必殺技発動！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
                return f"{self.name}の必殺技発動！{other.name}の体力は{other.hp}になった。"
            return f"{self.name}は味方に必殺技を使えない！"
            
        #ヒーラーの必殺技
        elif self.role == HEALER:
            if isinstance(other, Explorer):
                other.hp += 30
                self.mp -= 3
                return f"{self.name}の必殺技発動！{other.name}の体力は{other.hp}になった。"
            return f"{self.name}は敵を回復できない！"
            
        #魔法使いの必殺技
        elif self.role == MAGICIAN:
            if isinstance(other, Explorer):
                other.mp += 3
                return f"{self.name}の必殺技発動！{other.name}のMPは{other.mp}になった。"
            return f"{self.name}は敵にMP回復を施せない！"
        
        return f"{self.name}: 不明な役割のため、何も起こらなかった。"

    def reset(self):
        """初期状態に戻す"""
        self.__dict__ = copy.deepcopy(self._initial_state)

In [58]:
""" 
練習問題2) モンスタークラスを作成しよう。DORAGON、HARPY、WITCHの3タイプの必殺技を作ろう
"""

class Monster(Character):
    def __init__(self, name, hp, power, mp, breed):
        super().__init__(name, hp, power, mp)
        self.breed = breed
        self._initial_state = copy.deepcopy(self.__dict__)

    #スペシャルアタック
    def special_attack(self, other):
        
        #ドラゴンの必殺技
        if self.breed == DRAGON:
            #自分の攻撃力の2.5倍のダメージを与える
            other.hp -= self.power*2.5
            if other.hp <= 0:
                other.hp = 0
                self.mp -= 3
                return f"{self.name}のファイヤーブレス発動！{other.name}の体力は{other.hp}になった。\n{other.name}は{self.name}に倒された!"
            return f"{self.name}のファイヤーブレス発動！{other.name}の体力は{other.hp}になった。"
            
        #ハーピーの必殺技
        elif self.breed== HARPY:
            #自分の体力を40回復
            self.hp += 40
            self.mp -= 3
            return f"{self.name}の必殺技発動！{self.name}の体力は{self.hp}になった。"

        #ウィッチの必殺技
        elif self.breed == WITCH:
            #相手のmpを2奪い取り、自身のmpを2回復
            other.mp -= max(0, other.mp - 2)
            self.mp -= 1
            return f"{self.name}のドレイン発動！{other.name}のMPは{other.mp}になった。"

In [59]:
# --- ゲーム本体 (バトルマネージャー) --- 
class BattleManager:
    def __init__(self, party, enemy_team):
        self.party = party
        self.enemy_team = enemy_team
        self.turn = 1
        self.is_game_over = False

    def display_status(self):
        #全員のステータスを表示
        print("\n" + "="*40)
        print(f"--- ターン: {self.turn} ---")
        print("\n[勇者パーティ]")
        for i, member in enumerate(self.party):
            print(f" {i+1}. {member.name} ({member.role}): HP {member.hp}/{member.max_hp}, MP {member.mp}")
        
        print("\n[モンスター]")
        # 1体のみ表示される
        for i, monster in enumerate(self.enemy_team):
            print(f" {i+1}. {monster.name} ({monster.breed}): HP {monster.hp}/{monster.max_hp}, MP {monster.mp}")
        print("="*40 + "\n")

    def player_turn(self):
        #プレイヤーが操作するターン
        print("--- あなたのターン ---")
        
        for player in self.party:
            if not player.is_alive():
                continue

            print(f"\n[{player.name} の行動]")
            
            def select_target(targets, target_type):
                while True:
                    try:
                        print(f"ターゲットを選択してください:")
                        # 生きているターゲットのみ表示
                        alive_targets = [t for t in targets if t.is_alive()]
                        if not alive_targets:
                             print("ターゲットがいません！")
                             return None

                        for i, target in enumerate(alive_targets):
                            print(f" {i+1}. {target.name} (HP: {target.hp})")
                        
                        target_index = int(input("番号を入力 (1/...): ")) - 1
                        
                        if 0 <= target_index < len(alive_targets):
                            return alive_targets[target_index]
                        else:
                            print("無効な番号です。再入力してください。")
                    except ValueError:
                        print("数字を入力してください。")
            
            # ターゲットが存在しない場合はスキップ
            if not self.enemy_team and (player.role == HERO or player.role == MAGICIAN):
                 continue
            if not self.party and (player.role == HEALER or player.role == MAGICIAN):
                 continue

            while True:
                action = input(f"行動を選択 (a:通常攻撃 / s:必殺技): ").lower()
                
                if action == 'a':
                    target = select_target(self.enemy_team, "モンスター")
                    if target:
                        print(player.attack(target))
                        break
                
                elif action == 's':
                    if player.role == HERO:
                        target = select_target(self.enemy_team, "モンスター")
                    elif player.role == HEALER or player.role == MAGICIAN:
                        target = select_target(self.party, "パーティメンバー")
                    else:
                        print("その役割には必殺技がありません。")
                        continue
                    
                    if target:
                        print(player.special_attack(target))
                        break
                
                else:
                    print("無効な入力です。'a' または 's' を入力してください。")
            
            if self.check_game_status():
                return

    def enemy_turn(self):
        #モンスターが行動するターン
        print("\n--- 相手のターン ---")
        
        alive_players = [p for p in self.party if p.is_alive()]
        if not alive_players:
            return

        for monster in self.enemy_team:
            if not monster.is_alive():
                continue

            target = random.choice(alive_players)
            
            # モンスターのAI: 必殺技 or 通常攻撃
            if random.random() < 0.3 or \
               (monster.breed == HARPY and monster.hp < monster.max_hp * 0.5): # HPが低いハーピーは回復優先
                 print(monster.special_attack(target))
            else:
                print(monster.attack(target))

            if self.check_game_status():
                return

    def check_game_status(self):
        #勝敗のチェック
        alive_party = [p for p in self.party if p.is_alive()]
        alive_enemies = [m for m in self.enemy_team if m.is_alive()]
        
        if not alive_party:
            print("\n!!! 勇者パーティは全滅した。ゲームオーバー !!!")
            self.is_game_over = True
            return True
        
        if not alive_enemies:
            print("\n!!! モンスターをすべて倒した！勝利 !!!")
            self.is_game_over = True
            return True
        
        self.party = alive_party
        self.enemy_team = alive_enemies
        return False

    def start_battle(self):
        #戦闘の開始と実行
        enemy_name = self.enemy_team[0].name if self.enemy_team else "敵"
        print(f"{enemy_name}が立ちはだかる！バトル開始！")
        while not self.is_game_over:
            self.display_status()
            
            self.player_turn()
            if self.is_game_over: break
            
            self.enemy_turn()
            if self.is_game_over: break
            
            self.turn += 1
        
        print("\n--- 戦闘終了 ---")

In [60]:
import copy
import random
import sys

#Explorerの役割とMonsterの種族の決定
HERO = 'hero'
HEALER = 'healer'
MAGICIAN = 'magician'
DRAGON = 'dragon'
HARPY = 'harpy'
WITCH = 'witch'

if __name__ == "__main__":
    # 勇者パーティーの生成
    party_members = [
        Explorer("アーサー", 100, 15, 5, HERO),
        Explorer("リア", 80, 10, 8, HEALER),
        Explorer("ゼノ", 70, 20, 10, MAGICIAN),
    ]

    # --- モンスター隊の生成 (ランダムに1体を選択) ---
    MONSTER_TEMPLATES = {
        DRAGON: ("フレイム", 150, 20, 0),  # HP高め、高攻撃力
        HARPY: ("サイレン", 90, 15, 5),   # HP中、回復持ち
        WITCH: ("リリス", 110, 18, 10),   # HP中、MPドレイン持ち
    }

    # 3つの種族からランダムに1つ選ぶ
    random_breed = random.choice([DRAGON, HARPY, WITCH])
    name, hp, power, mp = MONSTER_TEMPLATES[random_breed]
    
    # 選択された1体のみを生成
    single_monster = Monster(name, hp, power, mp, random_breed)
    enemy_squad = [single_monster]
    
    # バトル開始
    try:
        game = BattleManager(party_members, enemy_squad)
        game.start_battle()
    except EOFError:
        print("\nゲームを終了します。")
    except Exception as e:
        print(f"\n予期せぬエラーが発生しました: {e}")
        sys.exit(1)

リリスが立ちはだかる！バトル開始！

--- ターン: 1 ---

[勇者パーティ]
 1. アーサー (hero): HP 100/100, MP 5
 2. リア (healer): HP 80/80, MP 8
 3. ゼノ (magician): HP 70/70, MP 10

[モンスター]
 1. リリス (witch): HP 110/110, MP 10

--- あなたのターン ---

[アーサー の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  a


ターゲットを選択してください:
 1. リリス (HP: 110)


番号を入力 (1/...):  1


アーサーがリリスを攻撃！リリスの体力は95になった。

[リア の行動]


...

アーサーがリリスを攻撃！リリスの体力は10になった。

[リア の行動]


行動を選択 (a:通常攻撃 / s:必殺技):  a


ターゲットを選択してください:
 1. リリス (HP: 10)


番号を入力 (1/...):  1


リアがリリスを攻撃！リリスの体力は0になった。
リリスはリアに倒された!

!!! モンスターをすべて倒した！勝利 !!!

--- 戦闘終了 ---
